Demand Predict

Basic Data Preparation

In [ ]:
from pandas import read_csv
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
dataset = read_csv('~/work/DemandPrediction/2016demand2017.csv')

In [ ]:
# manually specify column names
dataset.columns = ['date', 'quantity']
dataset.index.name = 'date'
dataset.head(5)

In [ ]:
from matplotlib import pyplot
# load dataset
#dataset = read_csv('pollution.csv', header=0, index_col=0)
values = dataset.values

In [ ]:

# plot each column
pyplot.figure()
pyplot.plot(values[:, 1])
pyplot.title(dataset.columns[1], y=0.5, loc='right')
pyplot.show()

Multivariate LSTM Forecast Model

LSTM Data Preparation:<br>
***framing the dataset as a supervised learning problem and normalizing the input variables.***

In [ ]:
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 

In [ ]:
# integer encode direction
encoder = LabelEncoder()
values[:,0] = encoder.fit_transform(values[:,0])

In [ ]:
values

In [ ]:
# ensure all data is float
values = values.astype('float32')

In [ ]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [ ]:
#1month
num_lags = 30
num_features=1
# frame as supervised learning
reframed = series_to_supervised(scaled, num_lags, 1)

In [ ]:
reframed.head(10)

In [ ]:
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
#print(reframed.head())

In [ ]:
values = reframed.values
values.shape

Define and Fit Model

In [ ]:
import math
# split into train and test sets
values = reframed.values
train_size = math.floor(values.shape[0] *0.6)
test_size = math.floor(values.shape[0]*0.2)
#n_train_day = 20
train = values[:train_size,:]
test = values[train_size:(train_size+test_size), :]
cross_validation = values[(train_size+test_size):, :]



In [ ]:
# split into input and outputs
n_obs = num_lags * num_features
train_X, train_y = train[:, :n_obs], train[:, -num_features]
test_X, test_y = test[:, :n_obs], test[:, -num_features]
cross_X, cross_y = cross_validation[:, :n_obs], cross_validation[:, -num_features]
print(train_X.shape, len(train_X), train_y.shape)

In [ ]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], num_lags, num_features))
test_X = test_X.reshape((test_X.shape[0], num_lags, num_features))
cross_X = cross_X.reshape((cross_X.shape[0], num_lags, num_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

In [ ]:
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Evaluate Model

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], num_lags*num_features))

In [ ]:
from numpy import concatenate
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -1:]), axis=1)
print(inv_yhat.shape)


In [ ]:
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
rmse = rmse* (1/100)

In [ ]:
print('Test RMSE: %.3f' % rmse)

In [ ]:
#save model
model.save("./model.h5")

In [ ]:
from keras.models import load_model
# load model
model = load_model('model.h5')
# summarize model.
model.summary()

In [ ]:
cross_validation
yhat = model.predict(cross_X)


In [ ]:
yhat

In [ ]:
#cross_validation.to_csv("cross.csv")

In [ ]:
import pandas
df = pandas.DataFrame(cross_validation)

In [ ]:
df.to_csv('~/work/Data/test.csv')

In [ ]:
cross_X.shape

In [ ]:
cross_X = cross_X.reshape((cross_X.shape[0], num_lags*num_features))
from numpy import concatenate
# invert scaling for forecast
inv_yhat = concatenate((yhat, cross_X[:, -1:]), axis=1)
print(inv_yhat.shape)


In [ ]:
inv_yhat

In [ ]:
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
inv_yhat

In [ ]:
#90 days 
num_lags = 90
num_features=1
# frame as supervised learning
reframed = series_to_supervised(scaled, num_lags, 1)
values = reframed.values
values = reframed.values
train_size = math.floor(values.shape[0] *0.6)
test_size = math.floor(values.shape[0]*0.2)
#n_train_day = 20
train = values[:train_size,:]
test = values[train_size:(train_size+test_size), :]
cross_validation = values[(train_size+test_size):, :]

# split into input and outputs
n_obs = num_lags * num_features
train_X, train_y = train[:, :n_obs], train[:, -num_features]
test_X, test_y = test[:, :n_obs], test[:, -num_features]
cross_X, cross_y = cross_validation[:, :n_obs], cross_validation[:, -num_features]
print(train_X.shape, len(train_X), train_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], num_lags, num_features))
test_X = test_X.reshape((test_X.shape[0], num_lags, num_features))
cross_X = cross_X.reshape((cross_X.shape[0], num_lags, num_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
#save model
model.save("./model-90days.h5")

In [ ]:
#90 days 
num_lags = 365
num_features=1
# frame as supervised learning
reframed = series_to_supervised(scaled, num_lags, 1)
values = reframed.values
values = reframed.values
train_size = math.floor(values.shape[0] *0.6)
test_size = math.floor(values.shape[0]*0.2)
#n_train_day = 20
train = values[:train_size,:]
test = values[train_size:(train_size+test_size), :]
cross_validation = values[(train_size+test_size):, :]

# split into input and outputs
n_obs = num_lags * num_features
train_X, train_y = train[:, :n_obs], train[:, -num_features]
test_X, test_y = test[:, :n_obs], test[:, -num_features]
cross_X, cross_y = cross_validation[:, :n_obs], cross_validation[:, -num_features]
print(train_X.shape, len(train_X), train_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], num_lags, num_features))
test_X = test_X.reshape((test_X.shape[0], num_lags, num_features))
cross_X = cross_X.reshape((cross_X.shape[0], num_lags, num_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
#save model
model.save("./model-365days.h5")